### 설명
* 이마트몰 밀키트 카테고리(ID: 6000217707) 에서 판매처/상품명/가격 스크래핑

## Code Assist
* python code AI assistant : claude
* prompt
```
https://emart.ssg.com/disp/category.ssg?dispCtgId=6000217707&sort=sale&page=
의 각 페이지의 <li class="mnemitem_grid_item"> 요소에서
'순서'라는 제목으로 순서값을 지정하고
판매처에 해당하는 <div class="mnemitem_tit "> 아래 <span class="mnemitem_goods_brand"> 값과
상품명에 해당하는 <div class="mnemitem_tit "> 아래 <span class="mnemitem_goods_tit"> 값과
가격에 해당하는 <div class="mnemitem_prd_per"> 아래 <em class="ssg_price"> 값
평점에 해당하는 <div class="mnemitem_review_score"> 아래 첫번째 <span class="review_text"> 값
리뷰수에 해당하는 <div class="mnemitem_review_score"> 아래 두번째 <span class="review_text"> 값
을 수집하는 파이썬 코드
```

## revision
* 1 : 이마트몰 밀키트 카테고리에서 판매처와 상품명, 가격 스크래핑
* 2 : 밀키트 카테고리 검색시 판매량 순으로 정렬 및 순위 지정. 한국식 음식으로 카테고리 분류
* 3
  * 평점, 리뷰수 수집 추가
  * 전처리 : '판매처'명이 "--" 인 경우 상품명에서 추출
  * 전처리 : 상품명에 무게를 뜻하는 'g'이 표기된 것이 많아 해당 무게값 제거

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

In [ ]:
final_csv_name = 'emart_mealkit_prod_salesranked'

In [ ]:
def get_product_info(page_num):
    """
    지정된 페이지에서 상품 정보(순서, 판매처, 상품명, 가격)를 수집하는 함수

    Args:
        page_num (int): 수집할 페이지 번호

    Returns:
        list: 상품 정보 (순서, 판매처, 상품명, 가격, 평점, 리뷰수) 튜플의 리스트
    """
    url = f"https://emart.ssg.com/disp/category.ssg?dispCtgId=6000217707&sort=sale&page={page_num}"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        products = []
        items = soup.find_all('li', class_='mnemitem_grid_item')

        # 페이지별 시작 순서 계산
        start_index = (page_num - 1) * len(items) + 1

        for idx, item in enumerate(items, start=start_index):
            try:
                # 판매처 찾기
                brand_elem = item.find('span', class_='mnemitem_goods_brand')
                brand = brand_elem.text.strip() if brand_elem else "--"

                # 상품명 찾기
                title_elem = item.find('span', class_='mnemitem_goods_tit')
                title = title_elem.text.strip() if title_elem else "--"

                # 가격 찾기
                price_elem = item.find('em', class_='ssg_price')
                price = price_elem.text.strip() if price_elem else "0"
                price = ''.join(filter(str.isdigit, price))  # 숫자만 추출

                # 평점과 리뷰 수 찾기
                review_scores = item.select('span.review_text')
                rating = review_scores[0].text.strip() if len(review_scores) > 0 else "0"
                review_count = review_scores[1].text.strip() if len(review_scores) > 1 else "0"
                review_count = ''.join(filter(str.isdigit, review_count))  # 숫자만 추출

                # 순서와 함께 저장
                products.append((idx, brand, title, price, rating, review_count))

            except Exception as e:
                print(f"상품 정보 추출 중 오류 (순서: {idx}): {str(e)}")
                continue

        return products

    except Exception as e:
        print(f"페이지 {page_num} 요청 중 오류 발생: {str(e)}")
        return []

In [ ]:
def scrape_multiple_pages(start_page, end_page):
    """
    여러 페이지의 상품 정보를 수집하는 함수

    Args:
        start_page (int): 시작 페이지 번호
        end_page (int): 끝 페이지 번호

    Returns:
        pandas.DataFrame: 수집된 상품 정보
    """
    all_products = []

    for page in range(start_page, end_page + 1):
        print(f"페이지 {page} 수집 중...")
        products = get_product_info(page)
        all_products.extend(products)

        # 서버 부하를 줄이기 위한 딜레이 (2초)
        time.sleep(2)

    # DataFrame 생성 및 중복 제거
    df = pd.DataFrame(all_products, columns=['순서', '판매처', '상품명', '가격', '평점', '리뷰수'])
    df = df.drop_duplicates()

    # 순서를 기준으로 정렬
    df = df.sort_values('순서')

    return df

In [ ]:
def main():
    """
    메인 실행 함수
    """
    try:
        start_time = datetime.now()
        print(f"수집 시작: {start_time}")

        # 1페이지부터 전체 12페이지까지 수집
        df = scrape_multiple_pages(1, 12)

        # 결과 저장
        output_file = final_csv_name + '-step1.csv'
        df.to_csv(output_file, index=False, encoding='utf-8-sig')

        # 통계 출력
        end_time = datetime.now()
        duration = end_time - start_time

        print(f"\n수집 완료!")
        print(f"시작 시간: {start_time}")
        print(f"종료 시간: {end_time}")
        print(f"소요 시간: {duration}")
        print(f"총 수집 상품 수: {len(df)}개")
        print(f"저장 파일명: {output_file}")

    except Exception as e:
        print(f"프로그램 실행 중 오류 발생: {str(e)}")

if __name__ == "__main__":
    main()

수집 시작: 2025-01-06 13:03:01.920739
페이지 1 수집 중...
페이지 2 수집 중...
페이지 3 수집 중...
페이지 4 수집 중...
페이지 5 수집 중...
페이지 6 수집 중...
페이지 7 수집 중...
페이지 8 수집 중...
페이지 9 수집 중...
페이지 10 수집 중...
페이지 11 수집 중...
페이지 12 수집 중...

수집 완료!
시작 시간: 2025-01-06 13:03:01.920739
종료 시간: 2025-01-06 13:03:53.941431
소요 시간: 0:00:52.020692
총 수집 상품 수: 943개
저장 파일명: emart_mealkit_prod_salesranked-step1.csv


## 전처리
* 상품명에서 무게를 나타내는 g 내용 제거
* 판매처명이 "--" 인 경우 상품명이 '['로 시작하는 경우 ']' 까지의 내용 사용

In [ ]:
# prompt: final_csv_name + '-step1.csv' 파일에서 상품명 내용 마지막에 'g' 문자가 있는 경우 그 앞에 있는 공백 문자부터 'g'까지 값 제거

try:
    # CSV 파일 읽기
    df = pd.read_csv(final_csv_name + '-step1.csv')

    # 상품명에서 'g' 문자를 찾아 처리
    def remove_weight(product_name):
        if isinstance(product_name, str) and 'g' in product_name:
            index = product_name.rfind('g')
            # 'g' 앞의 공백부터 'g'까지 제거
            return product_name[:product_name.rfind(' ', 0, index)].strip()
        return product_name

    df['상품명'] = df['상품명'].apply(remove_weight)

    # 결과를 새로운 CSV 파일로 저장
    df.to_csv(final_csv_name + '-step2.csv', index=False, encoding='utf-8-sig')
    print(f"상품명에서 'g' 값 제거가 완료되어 '{final_csv_name}-step2.csv' 파일로 저장되었습니다.")

except FileNotFoundError:
    print(f"{final_csv_name}-step1.csv' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

상품명에서 'g' 값 제거가 완료되어 'emart_mealkit_prod_salesranked-step2.csv' 파일로 저장되었습니다.


In [ ]:
# prompt: final_csv_name + '-step2.csv' 파일에서 판매처 값이 '--' 인 경우 상품명 내용 처음 문자가 '[' 인 경우 ']' 문자까지의 값을 판매처에 넣고 상품명에서는 그 내용 제거

import pandas as pd

try:
    # CSV 파일 읽기
    df = pd.read_csv(final_csv_name + '-step2.csv')

    # 판매처 값이 '--' 이고 상품명이 '['로 시작하는 경우 처리
    for index, row in df.iterrows():
        if row['판매처'] == '--' and isinstance(row['상품명'], str) and row['상품명'].startswith('['):
            end_bracket = row['상품명'].find(']')
            if end_bracket != -1:
                df.loc[index, '판매처'] = row['상품명'][1:end_bracket]
                df.loc[index, '상품명'] = row['상품명'][end_bracket + 1:].strip()

    # 결과를 새로운 CSV 파일로 저장
    df.to_csv(final_csv_name + '.csv', index=False, encoding='utf-8-sig')
    print(f"판매처 및 상품명 수정이 완료되어 '{final_csv_name}.csv' 파일로 저장되었습니다.")

except FileNotFoundError:
    print(f"{final_csv_name}-type2.csv 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

판매처 및 상품명 수정이 완료되어 'emart_mealkit_prod_salesranked.csv' 파일로 저장되었습니다.


In [ ]:
# prompt: emart_mealkit_prod_salesranked.csv 파일에서 상품명을 기준으로 한국식 음식으로 카테고리 분류 후
# emart_mealkit_prod_salesranked_with_type.csv 로 저장

def categorize_korean_food(product_name):
  """
  상품명을 기반으로 한국 음식 카테고리를 분류합니다.
  """
  product_name = str(product_name).lower() # 소문자 변환

  if any(keyword in product_name for keyword in ["쌀국수", "베트남식"]):
      return "베트남식"
  elif any(keyword in product_name for keyword in ["찌개", "김치찌개", "된장찌개", "부대찌개", "순두부찌개"]):
    return "찌개류"
  elif any(keyword in product_name for keyword in ["볶음밥", "비빔밥", "김밥", "잡채", "볶음"]):
    return "밥/면류"
  elif any(keyword in product_name for keyword in ["불고기", "갈비", "제육볶음", "돼지갈비", "닭갈비"]):
    return "고기류"
  elif any(keyword in product_name for keyword in ["전", "빈대떡", "파전", "김치전"]):
    return "전류"
  elif any(keyword in product_name for keyword in ["찜", "갈비찜", "닭볶음탕"]):
    return "찜류"
  elif any(keyword in product_name for keyword in ["국", "미역국", "된장국", "떡국"]):
    return "국류"
  elif any(keyword in product_name for keyword in ["구이", "삼겹살", "목살"]):
    return "구이류"
  elif any(keyword in product_name for keyword in ["닭", "닭볶음탕", "닭갈비"]):
    return "닭요리"
  elif any(keyword in product_name for keyword in ["해물", "생선", "회"]):
    return "해물/생선류"
  elif any(keyword in product_name for keyword in ["짬뽕", "짜장", "탕수육"]):
      return "중식"
  elif any(keyword in product_name for keyword in ["오꼬노미야끼", "야키소바", "라멘"]):
      return "일식"
  elif any(keyword in product_name for keyword in ["떡볶이", "순대", "튀김", "김말이"]):
      return "분식류"
  elif any(keyword in product_name for keyword in ["김치", "된장", "깍두기", "열무김치", "배추김치"]):
      return "기타한식류"
  else:
    return "기타"

try:
    # CSV 파일 읽기
    df = pd.read_csv(final_csv_name + '.csv')

    # 상품명에 대한 카테고리 분류
    df['카테고리'] = df['상품명'].apply(categorize_korean_food)

    # 결과를 새로운 CSV 파일로 저장
    df.to_csv(final_csv_name + '_with_type.csv', index=False, encoding='utf-8-sig')
    print(f"카테고리 분류가 완료되어 '{final_csv_name}_with_type.csv' 파일로 저장되었습니다.")

except FileNotFoundError:
    print(f"{final_csv_name}.csv 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

카테고리 분류가 완료되어 'emart_mealkit_prod_salesranked_with_type.csv' 파일로 저장되었습니다.
